In [1]:
using CSV, DataFrames, Statistics, Dates, Plots, LinearAlgebra, DecisionTree

In [2]:
using Pkg
Pkg.add("Plots")

    Updating registry at `C:\Users\Antoine\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Antoine\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Antoine\.julia\environments\v1.10\Manifest.toml`


In [5]:
trainData = CSV.read("../../data/train.csv", DataFrame)
testData = CSV.read("../../data/test.csv", DataFrame)
first(trainData, 5)

Row,annee,type,nombre_cylindres,cylindree,transmission,boite,consommation
,Int64,String31,Int64,String3,String15,String15,String31
1,2023,voiture_moyenne,8,"4,4",integrale,automatique,"13,8358823529412"
2,2020,VUS_petit,4,2,integrale,automatique,"9,80041666666667"
3,2021,voiture_compacte,6,"3,3",propulsion,automatique,"11,7605"
4,2023,voiture_deux_places,8,5,integrale,automatique,"13,0672222222222"
5,2022,voiture_moyenne,8,"4,4",integrale,automatique,"13,8358823529412"


In [6]:
trainData.consommation = parse.(Float64,replace.(trainData.consommation, "," => "."))
trainData.cylindree = parse.(Float64,replace.(trainData.cylindree, "," => ".")) 

testData.cylindree = parse.(Float64,replace.(testData.cylindree, "," => "."))

150-element Vector{Float64}:
 2.5
 2.5
 2.5
 2.0
 5.8
 5.0
 5.0
 2.4
 3.5
 5.2
 5.2
 5.9
 2.0
 ⋮
 1.6
 3.3
 5.0
 2.0
 2.0
 1.6
 2.0
 2.0
 3.0
 3.0
 1.5
 2.0

In [7]:
trainData[!,:volume_gaz] = trainData[!,:nombre_cylindres] .* trainData[!,:cylindree]

testData[!,:volume_gaz] = testData[!,:nombre_cylindres] .* testData[!,:cylindree]

150-element Vector{Float64}:
 10.0
 10.0
 10.0
  8.0
 46.4
 40.0
 40.0
  9.6
 21.0
 52.0
 52.0
 70.80000000000001
  8.0
  ⋮
  6.4
 19.799999999999997
 40.0
  8.0
  8.0
  6.4
  8.0
  8.0
 18.0
 18.0
  4.5
  8.0

In [21]:
function encode(data, column)
    for c in unique(data[!, column])
        data[!, Symbol(c)] = ifelse.(data[!, column] .== c, 1, 0)
    end
    return data
end


function encode_data(data)
    encoded_data = deepcopy(data)
    encoded_data = encode(encoded_data, :type)
    encoded_data = encode(encoded_data, :transmission)
    encoded_data = encode(encoded_data, :boite)
    return encoded_data
end

function removeRows(data)
    return select!(data, Not([:type, :transmission, :boite]))
end

function normalize_column(data, column)
    data[!, column] = (data[!, column] .- mean(data[!, column])) ./ std(data[!, column])
    return data
end

function norm_cols(data)
    normalized_data = deepcopy(data)
    normalized_data = normalize_column(normalized_data, :cylindree)
    normalized_data = normalize_column(normalized_data, :volume_gaz)
    normalized_data = normalize_column(normalized_data, :nombre_cylindres)
    normalized_data = normalize_column(normalized_data, :annee)

    return normalized_data
end

norm_cols (generic function with 1 method)

In [23]:
encoded_train = encode_data(trainData)
filtered_train = removeRows(encoded_train)
normed_train = norm_cols(filtered_train)

encoded_test = encode_data(testData)
filtered_test = removeRows(encoded_test)
normed_test = norm_cols(filtered_test)

X_train =  Matrix(normed_train[:, Not(:consommation)])
y_train = normed_train.consommation
X_test = Matrix(normed_test)

150×24 Matrix{Float64}:
 -1.53728  -0.769722  -0.372358    …  0.0  1.0  0.0  0.0  0.0  1.0  0.0
 -1.53728  -0.769722  -0.372358       0.0  1.0  0.0  0.0  0.0  0.0  1.0
 -1.53728  -0.769722  -0.372358       0.0  1.0  0.0  0.0  0.0  0.0  1.0
 -1.53728  -0.769722  -0.739816       0.0  0.0  1.0  0.0  0.0  0.0  1.0
 -1.53728   1.45063    2.05287        0.0  0.0  0.0  1.0  0.0  1.0  0.0
 -1.53728   1.45063    1.46493     …  0.0  0.0  0.0  1.0  0.0  0.0  1.0
 -1.53728   1.45063    1.46493        0.0  0.0  0.0  1.0  0.0  1.0  0.0
 -1.53728  -0.769722  -0.44585        0.0  0.0  1.0  0.0  0.0  0.0  1.0
 -1.53728   0.340454   0.362559       0.0  0.0  0.0  0.0  1.0  0.0  1.0
 -1.53728   2.56081    1.61192        0.0  0.0  0.0  0.0  1.0  1.0  0.0
 -1.53728   2.56081    1.61192     …  0.0  0.0  0.0  0.0  1.0  1.0  0.0
 -1.53728   3.67098    2.12636        0.0  0.0  0.0  1.0  0.0  0.0  1.0
 -1.53728  -0.769722  -0.739816       0.0  1.0  0.0  0.0  0.0  0.0  1.0
  ⋮                                ⋱    

In [24]:
size(X_train), size(y_train), size(X_test)

((396, 23), (396,), (150, 24))

In [25]:
function decisionTree_remise(X_train ,y_train,  X_test)
    model = model = RandomForestRegressor(n_subfeatures=12, n_trees=600,min_samples_leaf=1,min_purity_increase=0.0, max_depth=10, min_samples_split=6)
    DecisionTree.fit!(model, X_train, y_train)
    ychap = DecisionTree.predict(model, X_test)
    return ychap
end

decisionTree_remise (generic function with 2 methods)

In [26]:
res = decisionTree_remise(X_train ,y_train,  X_test)
testData.consommation = res

150-element Vector{Float64}:
  9.473709862210896
  9.388619130708847
  9.637982768023718
  8.117922197805113
 14.841102148286291
 14.46397647414041
 14.87156971808104
  8.330211045365889
 10.929659373434125
 14.623880292370123
 14.623880292370123
 15.200416195377738
  9.178613370531854
  ⋮
  7.475400375281856
 11.095090777999134
 13.455020988443795
  8.846698322277618
  9.051720140931877
  7.38983181229766
  9.320360961205001
  8.831840302020254
 11.531254966047237
 10.937304671726238
  7.980609070002455
  9.045332035693153

In [1]:
id = 1:150

df_pred = DataFrame(id=id, consommation=res)

CSV.write("benchmark5.csv", df_pred)

LoadError: UndefVarError: `res` not defined